# Import

In [7]:
import numpy as np
import pandas as pd
import random
import re

# Library Bantuan
from libraryBantuan.katla import Katla

# Baca Data

In [6]:
daftar_kata = list(pd.read_csv('lima huruf.csv', header=None)[0])
print(f"Banyak Kata : {len(daftar_kata)}")

Banyak Kata : 6311


# Fungsi nebak

! = benar <br>
? = ada tapi salah tempat <br>
\* = tidak ada huruf tersebut


## Class Katla

## Class GameKatla

In [4]:
class GameKatla():
    def __init__(self, daftar_kata):
        self.jumlah_tebakan = 0
        self.MENANG = "!!!!!"
        self.__jawaban = random.choice(daftar_kata)


    def tebak(self, tebakan):
        self.jumlah_tebakan += 1
        respon_pola = self.__evaluasi_tebakan(tebakan)

        if respon_pola == self.MENANG:                  # Sudah sama maka menang
            return ("selesai", self.jumlah_tebakan)
        if self.jumlah_tebakan == 6:                    # Sudah habis batas tebak dan masih salah
            return ("selesai", 10)
        

    def __evaluasi_tebakan(self, tebakan):
        list_jawaban = []
        respon_pola = ['*']*5
        for i in range(5):
            if self.__jawaban[i] == tebakan[i]:
                respon_pola[i] = '!'
            else:
                list_jawaban.append(self.__jawaban[i])
        
        for i in range(5):
            if respon_pola[i] == '!':
                continue
            if tebakan[i] in list_jawaban:
                respon_pola[i] = '?'
                list_jawaban.remove(tebakan[i])
        return ''.join(respon_pola)

        

# Program Utama

In [5]:
katla = Katla(daftar_kata)

katla.tebakan_selanjutnya("semua", "***??")
katla.tebakan_selanjutnya("uktab", "?**??")
katla.tebakan_selanjutnya("banyu", "!!!*!")

'bandu'